In [1]:
import os, requests
import matplotlib.pyplot as plt
import nglview as nv
import MDAnalysis as mda
from ipywidgets import Output, HBox
import pandas as pd
import numpy as np

# Delete log files and results
for file in os.listdir():
    if not file.endswith(('ipynb','png','pdb','xtc')):
        print("rm", file)
        os.remove(file)

<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **MD_id**: PDB code of the protein structure (e.g. [A01M6](https://mmb-dev.mddbr.eu/#/id/A01M6/overview))

In [6]:
MD_id = 'A01M6'
steps = 1500

# common arguments for all the components
input_top_path = f'{MD_id}.pdb'
input_traj_path = f'{MD_id}.xtc'
lipid_sel = '(resname DPPC and name P8)'

<a id="fetch"></a>
***
## Fetching structure and trajectory

In [8]:
# Check if PDB and XTC files already exist
if not (os.path.exists(f"{MD_id}.pdb") and os.path.exists(f"{MD_id}.xtc")):
    # Download PDB
    headers = {'accept': 'chemical/x-pdb'}
    url = f"https://mmb-dev.mddbr.eu/api/rest/v1/projects/{MD_id}/files/structure.pdb"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        pdb_content = response.text
        # Save the PDB content to a file
        with open(f"{MD_id}.pdb", 'w') as pdb_file:   
            pdb_file.write(pdb_content)
        print(f"Saved PDB file")
    else:
        print(f"Error for the PDB file: {response.status_code}")
        
    # Download XTC
    headers2 = {'accept': 'application/octet-stream'}
    url2= f'https://mmb-dev.mddbr.eu/api/rest/v1/projects/{MD_id}/files/trajectory.xtc'
    response2 = requests.get(url2, headers=headers2)
    if response2.status_code == 200:
        xtc_content = response2.content
        with open(f"{MD_id}.xtc", 'wb') as xtc_file:
            xtc_file.write(xtc_content)
        print(f"Saved XTC file")
    else:
        print(f"Error for the XTC file: {response2.status_code}")

<a id="membrane"></a>
***
## Membrane identification

Membrane identification...<br>
***
**Building Blocks** used:
 - [FatslimMembranes](https://biobb-mem.readthedocs.io/en/latest/fatslim.html#module-fatslim.fatslim_membranes) from **biobb_mem.fatslim.fatslim_membranes**
***

In [17]:
from biobb_mem.fatslim.fatslim_membranes import fatslim_membranes

prop = {
    'selection': lipid_sel,
    'cutoff': 2.2
}
fatslim_membranes(input_top_path=input_top_path,
                  input_traj_path=input_traj_path,
                  output_ndx_path="leaflets.ndx", #"../test/reference/fatslim/leaflets.ndx",
                  properties=prop)

2024-11-29 17:09:27,184 [MainThread  ] [INFO ]  Module: biobb_mem.fatslim.fatslim_membranes Version: 5.0.2
2024-11-29 17:09:27,184 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_8a162505-c405-45e0-922a-932d7c86f0fb directory successfully created
2024-11-29 17:09:27,187 [MainThread  ] [INFO ]  Copy: A01M6.pdb to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_8a162505-c405-45e0-922a-932d7c86f0fb
2024-11-29 17:09:27,753 [MainThread  ] [INFO ]  Copy: A01M6.xtc to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_8a162505-c405-45e0-922a-932d7c86f0fb
2024-11-29 17:09:27,966 [MainThread  ] [INFO ]  gmx editconf -f /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_8a162505-c405-45e0-922a-932d7c86f0fb/A01M6.pdb -o d1ab93be-f04b-4478-9b20-e386739ced82/output.gro -box 147.3093 147.3093 177.3872 ; fatslim membranes -n e79f9cfc-108f-4972-9af1-5705c8ef5965/headgroups.ndx -c d1ab93be-f04b-4478-9b20-e386739ced82/output.g

0

In [18]:
from biobb_mem.fatslim.fatslim_membranes import parse_index
parse_index??

Signature: parse_index(ndx)
Source:   
def parse_index(ndx):
    """
    Parses a GROMACS index file (.ndx) to extract leaflet groups.

    Args:
        ndx (str): Path to the GROMACS index file (.ndx).
    Returns:
        dict: A dictionary where keys are group names and values are lists of integers representing atom indices.
    """

    # Read the leaflet.ndx file
    with open(ndx, 'r') as file:
        leaflet_data = file.readlines()

    # Initialize dictionaries to store leaflet groups
    leaflet_groups = {}
    current_group = None

    # Parse the leaflet.ndx file
    for line in leaflet_data:
        line = line.strip()
        if line.startswith('[') and line.endswith(']'):
            current_group = line[1:-1].strip()
            leaflet_groups[current_group] = []
        elif current_group is not None:
            leaflet_groups[current_group].extend(map(int, line.split()))
    return leaflet_groups
File:      ~/repo/biobb_mem/biobb_mem/fatslim/fatslim_membranes.py
Typ

In [ ]:
leaflet_groups = parse_index("leaflets.ndx")
u = mda.Universe(input_top_path, input_traj_path)
# Convert atoms list to resnums (nglview uses cannot use resindex)
top_resn = u.atoms[np.array(leaflet_groups['membrane_1_leaflet_1'])-1].residues.resnums    
bot_resn = u.atoms[np.array(leaflet_groups['membrane_1_leaflet_2'])-1].residues.resnums    
mem_resn = np.concatenate((top_resn, bot_resn))

# Visualize the system with NGLView
view = nv.show_mdanalysis(u)
view.clear_representations()
view.add_point(selection='DPPC', color='white')                         # all lipids
view.add_point(selection=", ".join(map(str, mem_resn)), color='blue')   # change lipids in membrane to blue
display(view)

NGLWidget(max_frame=10000)

<a id="density"></a>
***
## Density profile

Calculate density...<br>
***
**Building Blocks** used:
 - [CpptrajDensity](https://biobb-mem.readthedocs.io/en/latest/ambertools.html#module-ambertools.cpptraj_density) from **biobb_mem.ambertools.cpptraj_density**
***

In [9]:
from biobb_mem.ambertools.cpptraj_density import cpptraj_density

prop = {
    'mask': '::M ::A,B',
    'steps': steps,
}
cpptraj_density(input_top_path=input_top_path,
            input_traj_path=input_traj_path,
            output_cpptraj_path='density_mask.dat',
            properties=prop)

2024-11-29 16:38:03,439 [MainThread  ] [INFO ]  Module: biobb_mem.ambertools.cpptraj_density Version: 5.0.2
2024-11-29 16:38:03,440 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_a80af771-05d7-4209-850c-b5e82c23ef43 directory successfully created
2024-11-29 16:38:03,442 [MainThread  ] [INFO ]  Copy: A01M6.pdb to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_a80af771-05d7-4209-850c-b5e82c23ef43
2024-11-29 16:38:04,007 [MainThread  ] [INFO ]  Copy: A01M6.xtc to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_a80af771-05d7-4209-850c-b5e82c23ef43
2024-11-29 16:38:04,008 [MainThread  ] [INFO ]  cpptraj -i 663cd2e9-f7dd-43f5-8f11-b329bed556fc/instructions.in

2024-11-29 16:38:04,164 [MainThread  ] [INFO ]  Executing: cpptraj -i 663cd2e9-f7dd-43f5-8f11-b329bed556fc/instructions.in...
2024-11-29 16:38:04,164 [MainThread  ] [INFO ]  Exit code: 0
2024-11-29 16:38:04,164 [MainThread  ] [INFO ]  
CPPTRAJ: Trajectory Analysis. 

0

### Visualization

In [10]:
def plot_and_view(den,view,labels,axis='z'):
    axis = f'#{axis.upper()}'
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    cls = den.columns[1:]
    cls = den.columns[1:]
    for i,cl in enumerate(cls):
        if i%2==0:
            ax.plot(den[axis], den[cl], label=labels[i//2])
            cl0 = cl
        else:
            ax.fill_between(den[axis], den[cl0] - den[cl], den[cl0] + den[cl],label='SD',color='red')
    ax.set_title('Mass Density');ax.set_xlabel('Å')
    ax.set_ylabel('Density'); ax.legend()

    # Put plot in a widget and display both widgets in a row
    out = Output()
    with out: plt.show()
    box = HBox([out,view]); display(box)
    view._remote_call('setSize', target='Widget', args=['100%', '100%'])
    view.control.spin([0, 1, 0], -1.57) 

In [11]:
# Mask
view = nv.show_file(input_top_path)
den = pd.read_csv(f'density_mask.dat', sep='\s+')
plot_and_view(den,view,['Membrane','Protein'])
view.update_ball_and_stick(color='blue')
view.update_cartoon(color='orange')

<a id="leaflets"></a>
***
## Leafleats assignation

Lyphilic is faster that fatslim, it algorithm is simpler, assumes a plane membrane, it let you do the assignation of leafleats for every frame and use this information for analysis like flip-flop <br>
***
**Building Blocks** used:
 - [LPPAssignLeaflets](https://biobb-mem.readthedocs.io/en/latest/lipyphilic_biobb.html) from **from biobb_mem.lipyphilic_biobb.lpp_assign_leaflets**
***

![DPPC](../html/DPPC.png "DPPC")

In [ ]:
from biobb_mem.lipyphilic_biobb.lpp_assign_leaflets import lpp_assign_leaflets

prop = {
    'lipid_sel': lipid_sel,
    'steps': steps
}
lpp_assign_leaflets(input_top_path=input_top_path,
                    input_traj_path=input_traj_path,
                    output_leaflets_path= 'leaflets_data.csv',
                    properties=prop)

/home/rchaves/miniforge3/envs/biobb_mem/lib/python3.10/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


2024-11-29 16:55:38,667 [MainThread  ] [INFO ]  Module: biobb_mem.lipyphilic_biobb.lpp_assign_leaflets Version: 5.0.2
2024-11-29 16:55:38,668 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_80a36a37-2e1f-4899-831d-e70d77390f4a directory successfully created
2024-11-29 16:55:38,671 [MainThread  ] [INFO ]  Copy: A01M6.pdb to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_80a36a37-2e1f-4899-831d-e70d77390f4a
2024-11-29 16:55:39,264 [MainThread  ] [INFO ]  Copy: A01M6.xtc to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_80a36a37-2e1f-4899-831d-e70d77390f4a
2024-11-29 16:55:39,630 [MainThread  ] [INFO ]  Removed: ['/home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_80a36a37-2e1f-4899-831d-e70d77390f4a']
2024-11-29 16:55:39,631 [MainThread  ] [INFO ]  


0

### Visualization

In [4]:
from biobb_mem.lipyphilic_biobb.lpp_assign_leaflets import display_nglview
view = display_nglview(input_top_path, 'leaflets_data.csv')
view.control.spin([0, 1, 0], -1.57)
view

NGLWidget()

<a id="channel"></a>
***
## Channel

Chanel... <br>
***
**Building Blocks** used:
 - [MDAHole](https://biobb-mem.readthedocs.io/en/latest/mdanalysis_biobb.html#mdanalysis_biobb.mda_hole.MDAHole) from **from biobb_mem.mdanalysis_biobb.mda_hole**
***

In [3]:
from biobb_mem.mdanalysis_biobb.mda_hole import mda_hole

prop = {
    'select': 'protein',
    'steps': steps,
    'sample': 0.1
}
mda_hole(input_top_path=input_top_path,
         input_traj_path=input_traj_path,
         output_hole_path='hole.vmd',
         properties=prop)

2024-11-29 11:25:08,104 [MainThread  ] [INFO ]  Module: biobb_mem.mdanalysis_biobb.mda_hole Version: 5.0.2
2024-11-29 11:25:08,105 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_fc75b842-dae6-4ad5-b6bb-05a3dbfd5889 directory successfully created
2024-11-29 11:25:08,107 [MainThread  ] [INFO ]  Copy: A01M6.pdb to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_fc75b842-dae6-4ad5-b6bb-05a3dbfd5889


/home/rchaves/miniforge3/envs/biobb_wf_mem/lib/python3.11/site-packages/MDAnalysis/analysis/hole2/__init__.py:47: DeprecationWarning: Deprecated in version 2.8.0
MDAnalysis.analysis.hole2 is deprecated in favour of the MDAKit madahole2 (https://www.mdanalysis.org/mdahole2/) and will be removed in MDAnalysis version 3.0.0
  warnings.warn(wmsg, category=DeprecationWarning)


2024-11-29 11:25:08,683 [MainThread  ] [INFO ]  Copy: A01M6.xtc to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_fc75b842-dae6-4ad5-b6bb-05a3dbfd5889


/home/rchaves/miniforge3/envs/biobb_wf_mem/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/rchaves/miniforge3/envs/biobb_wf_mem/lib/python3.11/site-packages/MDAnalysis/lib/util.py:575: DeprecationWarning: This method is deprecated as of MDAnalysis version 2.7.0 and will be removed in version 3.0.0. Please use shutil.which instead.
  warnings.warn(wmsg, DeprecationWarning)


2024-11-29 11:25:23,842 [MainThread  ] [INFO ]  Removed: ['/home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_fc75b842-dae6-4ad5-b6bb-05a3dbfd5889']
2024-11-29 11:25:23,842 [MainThread  ] [INFO ]  


0

In [4]:
#%conda install -c conda-forge vmd -y # si se instala en el entorno conda da problemas de clobber y jupyter no arranca
#%conda install libsqlite --force-reinstall -y # vmd breaks it

In [5]:
# TODO: cargar más frame (lento pero se resolvería con H5MD) y mirar por que algunos caneles salen torcidos
vmd_script = f"""
mol addfile {input_traj_path} type xtc first 0 last 1 step {steps} waitfor all
animate delete beg 0 end 0
source hole.vmd
"""

# Join the contents and write to a new file
with open('vmd_script.vmd', 'w') as fl:
    fl.write(vmd_script)

!vmd -pdb {input_top_path} -e vmd_script.vmd

/home/rchaves/miniforge3/envs/biobb_wf_mem/lib/vmd_LINUXAMD64: /home/rchaves/miniforge3/envs/biobb_wf_mem/lib/./libGL.so.1: no version information available (required by /home/rchaves/miniforge3/envs/biobb_wf_mem/lib/vmd_LINUXAMD64)
Info) VMD for LINUXAMD64, version 1.9.3 (November 30, 2016)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 16 CPUs detected.
Info)   CPU features: SSE2 AVX AVX2 FMA 
Info) Free system memory: 56GB (90%)
Info) No CUDA accelerator devices available.
Warning) Detected X11 'Composite' extension: if incorrect display occurs
Warning) try disabling this X server option.  Most Op